In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from statistics import mean

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns

from himodule.custom_classes import NasaDataset, LossAndMetric, WindowedLoader, CAE
from himodule.ae_metrics import MAPE
from himodule.normalisation import StandardScaler, MinMaxScaler
from himodule.secondary_funcs import save_object, check_path, split_dataset, \
    seed_everything, split_anomaly_normal, split_anomaly_normal23

from collections import defaultdict

import os
from typing import Tuple

sns.set_theme(style='whitegrid', font_scale=1.2)

In [ ]:
# Check for GPU availability

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'{device=}')

### Training

In [ ]:
def calculate_window_size(window_size: int, kernel: int, stride: int):
    return (window_size - kernel) // stride + 1

def calculate_kernel_size(new_window_size: int, last_window_size: int, stride: int):
    return last_window_size - (new_window_size - 1) * stride

In [ ]:
SEED = 37
BATCH_SIZE = 64
window_size = 30

# Whole dataset loading
train_dataset = NasaDataset('../datasets/clean_train_data.csv')
# Separate normal and anomaly data
train_dataset, anomaly_dataset = split_anomaly_normal23(train_dataset)
seed_everything(SEED)
# Get datasets for training and validation
train_dataset, val_dataset = split_dataset(train_dataset, test_size=0.3)

# Test dataset loading
test_dataset = NasaDataset('../datasets/clean_test_data.csv')

scaler = MinMaxScaler()
try:
    norm_name = repr(scaler).split(' ', maxsplit=2)[0].split('.')[-1]
except IndexError:
    norm_name = 'no_scaling'
for idx, dtset in enumerate((train_dataset, val_dataset, test_dataset, anomaly_dataset)):
    dtset.to(device)
    if scaler:
        if idx == 0:
            scaler.fit(dtset.dataset)
        dtset.dataset = scaler.transform(dtset.dataset)

# Save trained scaler to use it in another files
scaler_path = os.path.join('../scalers/', f'{norm_name}.pkl')
if scaler and not os.path.exists(scaler_path):
    save_object(scaler, scaler_path)

seed_everything(SEED)
train_loader = WindowedLoader(train_dataset, batch_size=BATCH_SIZE, window_size=window_size, for_conv=True)
seed_everything(SEED)
val_loader = WindowedLoader(train_dataset, batch_size=BATCH_SIZE, window_size=window_size, for_conv=True)

seed_everything(SEED)
test_loader = WindowedLoader(val_dataset, batch_size=BATCH_SIZE, window_size=window_size, for_conv=True)

seed_everything(SEED)
anomaly_loader = WindowedLoader(anomaly_dataset, batch_size=BATCH_SIZE, window_size=window_size, for_conv=True)

print(f'Train: {len(train_dataset)}\nValidation: {len(val_dataset)}\nTest: {len(test_dataset)}')
print(f'Anomaly: {len(anomaly_dataset)}')

# Model creating
params_dct = {
    'conv_kernel': 3,
    'conv_stride': 1,
    'pool_kernel': 2,
    'pool_stride': 1,
    'unconv_stride': 1
}

conv_dct = {'kernel': params_dct['conv_kernel'], 'stride': params_dct['conv_stride']}
pool_dct = {'kernel': params_dct['pool_kernel'], 'stride': params_dct['pool_stride']}

window_sizes = [window_size]
for dct in (conv_dct, pool_dct, conv_dct, pool_dct):
    window_sizes.append(calculate_window_size(window_sizes[-1], **dct))

input_shape = train_dataset.get_input_shape()
layers_sizes = (input_shape*2,
                input_shape*4,
                window_sizes[-1]*input_shape*4,
                window_sizes[-1]*input_shape*4 // 8,
                window_sizes[-1]*input_shape*4 // 16)

kernels = list()
new_window_size = window_sizes[-1]
for last_window_size in window_sizes[-3::-2]:
    kernels.append(calculate_kernel_size(new_window_size, last_window_size, params_dct['unconv_stride']))
    new_window_size = last_window_size
params_dct.update({'unconv_kernels': kernels})

seed_everything(SEED)
model_cae = CAE(input_channels=input_shape, layers=layers_sizes, **params_dct).to(device)
loss_func = nn.MSELoss()
metric_func = MAPE()

transform = torch.transpose
transform_dct = {'dim0': 1, 'dim1': 2}
get_loss_and_metric = LossAndMetric(loss_func, metric_func, scaler,
                                    transform=transform, transform_dct=transform_dct)
get_loss_and_metric_anomaly = LossAndMetric(nn.MSELoss(reduction='none'), metric_func, scaler,
                                            transform=transform, transform_dct=transform_dct)

optimiser = optim.AdamW(model_cae.parameters(),
                       lr=1e-5)
optimiser_name = repr(optimiser).split(' ', maxsplit=1)[0]

In [ ]:
def evaluate_model(model: nn.Module, loader: DataLoader) -> Tuple[list, list]:

    '''Returns losses and metrics of trained model.'''

    with torch.no_grad():
        losses = list()
        metrics = list()
        for dta in loader:
            sample = dta['sensors']
            sample = sample.to(device)
            _, reconstruction = model(sample)

            loss, metric = get_loss_and_metric(reconstruction, sample)

            losses.append(loss.item())
            metrics.append(metric.item())
    return losses, metrics


def plot_history(loss_history_df: pd.DataFrame, metric_history_df: pd.DataFrame,
                 test_losses: float, test_metrics: float,
                 ylabel_loss: str = None, ylabel_metric: str = None, save_path: str = None):
    
    '''Makes plot with 4 axes: training losses and metrics, test losses, validation losses and metrics, test metrics.'''

    loss_history_df = loss_history_df.melt(ignore_index=False).iloc[1:]
    metric_history_df = metric_history_df.melt(ignore_index=False).iloc[1:]
    fig = plt.figure(layout="constrained")
    fig.set_size_inches(10, 10)

    gs = GridSpec(2, 5, figure=fig)
    ax1 = fig.add_subplot(gs[0, 0:4])
    ax2 = fig.add_subplot(gs[0, 4])
    
    ax3 = fig.add_subplot(gs[1, 0:4])
    ax4 = fig.add_subplot(gs[1, 4])

    sns.lineplot(data=loss_history_df,
                x=loss_history_df.index,
                y='value',
                hue='variable',
                ax=ax1)
    ax1.set_ylabel(ylabel_loss)
    ax1.set_ylim(0, np.percentile(loss_history_df['value'].values, 99))

    sns.boxplot(x=['test_loss']*len(test_losses),
                y=test_losses,
                color='g',
                ax=ax2)
    ax2.set_ylabel(None)

    sns.lineplot(data=metric_history_df,
                x=metric_history_df.index,
                y='value',
                hue='variable',
                ax=ax3)
    ax3.set_ylabel(ylabel_metric)

    sns.boxplot(x=['test_metric']*len(test_metrics),
                y=test_metrics,
                color='g',
                ax=ax4)
    ax4.set_ylabel(None)

    plt.tight_layout()
    if save_path:
        plt.savefig(save_path)
    plt.show()

In [ ]:
epochs = 100
history = list()

# Model training on normal data only

for epoch in range(epochs):
    train_losses = list()
    train_metrics = list()
    for dta in train_loader:
        sample = dta['sensors']
        sample = sample.to(device)
        _, reconstruction = model_cae(sample)

        loss, metric = get_loss_and_metric(reconstruction, sample)

        optimiser.zero_grad()
        loss.backward()
        optimiser.step()

        train_losses.append(loss.item())
        train_metrics.append(metric.item())
    
    val_losses, val_metrics = evaluate_model(model_cae, val_loader)
    
    train_loss, val_loss = mean(train_losses), mean(val_losses)
    train_metrics, val_metrics = mean(train_metrics), mean(val_metrics)
    history.append((epoch, train_loss, val_loss, train_metrics, val_metrics))
    if (epoch + 1) % 10 == 0 or epoch == epochs - 1:
        print(f'{epoch+1:>3}/{epochs:>3}: {train_loss=:.4f}, {val_loss=:.4f}, {train_metrics=:.4f}%, {val_metrics=:.4f}%')

test_losses, test_metrics = evaluate_model(model_cae, test_loader)
test_loss = mean(test_losses)
test_metric = mean(test_metrics)
print(f'\n{test_loss=:.4f}, {test_metric=:.4f}%')


#-----------------------------------------------------------#
# Model saving
name = str(window_size)
models_path = os.path.join('../Models/cae/', str(len(layers_sizes)+1))
plot_path = os.path.join('../Plots/cae/', str(len(layers_sizes)+1))

if True:
    check_path(models_path)
    torch.save(model_cae.state_dict(), os.path.join(models_path, f'{name}.pth'))


if True:
    # Make plots to evaluate model performance

    columns = ('epoch', 'train_loss', 'val_loss', 'train_metric', 'val_metric')
    total_history_df = pd.DataFrame(history, columns=columns).set_index('epoch')

    check_path(plot_path)

    loss_history_df = total_history_df.loc[:,('train_loss', 'val_loss')]
    metric_history_df = total_history_df.loc[:,('train_metric', 'val_metric')]
    plot_history(loss_history_df, metric_history_df, test_losses, test_metrics,
                ylabel_loss='MSE', ylabel_metric='MAPE', save_path=os.path.join(plot_path, f'{name}.png'))

In [ ]:
model_cae.eval()